In [ ]:
'''
conceptual notes
    goal is to create script that takes in a message and encrypts it using the rlce encryption method
    
    for the sake of simplicity, this code works with a binary matrix defined by GF(2)

    definitions
        m - message vector of length k
        n - codeword length
        Gs - generator matrix k X n
        t - number of errors that linear code can correct
        S - non singular matrix, k X k
        P - permutation matrix, n X n
        r - number of random columns added
        public key - how messages are encrypted; G = S * Gs * P
        private key - Gs
        codeword - message embedded into generator matrix
        ciphertext - codeword + some errors to further hide it; decrypted with the private key

    steps
        1. create random generator matrix Gs to create linear code
        2. add r number of columns to Gs to create G1, to hide Gs
        3. create matrix A to scramble the COLUMNS of G1 in blocks (chunks)
        4. generate random invertible matrix S to scramble the ROWS of G1
        5. generate matrix P to shuffle columns of G1
        6. generate public key
        7. pass back private key

'''

# note to self - write out steps in rlce encryption, apply functions

import numpy as np
import galois

GF2 = galois.GF(2) # defining GF(2) finite field

# function to make random matrix with specified num of rows/columns, within GF(2)
def randomMatrix(rows, cols):
    return np.random.randint(0,2, size=(rows, cols))

# function to perform matrix multiplacation and keep answers within GF(2)
def formCodeword(m, G):
    return np.mod(np.dot(m, G), 2)

# function to make inverse of GF(2) matrix 
def inverseMatrix(matrix):
    m = GF2(matrix) # converting numpy matrix to datatype applicable to galois library inverse operation
    m_inv = m.inverse()
    return np.array(m_inv, dtype=int)

# function to decrypt matrix
def decodeMatrix(ciphertext, G):
    G_inv = GF2.inverse(G)
    return np.mod(np.dot(ciphertext, G_inv), 2)
